In [30]:
import numpy as np
import pandas as pd


In [3]:
import seaborn as sns
datasets = sns.load_dataset('titanic', engine="python")

In [4]:
datasets.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### apply 함수
- lambda 식과 같이 사용한다
- 행이나 열 단위로 복잡한 데이터 가공이 필요한 경우
- apply 함수는 인자로 함수를 넘겨 받을 수 있다

In [4]:
def get_square(a) :
    return a**2

In [5]:
print("제곱근:", get_square(2))

제곱근: 4


In [6]:
# 위 코드를 람다식으로 바꾼다면?
lambda_square = lambda a : a**2

In [7]:
print("제곱근:", lambda_square(2))

제곱근: 4


In [8]:
np.random.seed(100)

In [85]:
sampleDF = pd.DataFrame(np.random.randint(0,10,(6,4)))
sampleDF.columns = ['A','B','C','D']
sampleDF.index = pd.date_range("20210302", periods=6)

In [16]:
sampleDF

,A,B,B,D
2021-03-02,5,1,8,1
2021-03-03,5,4,2,8
2021-03-04,3,5,0,9
2021-03-05,3,6,3,4
2021-03-06,7,6,3,9
2021-03-07,0,4,4,5


In [17]:
# 각 행의 column에 대해서 최댓값 - 최솟값을 구해서 새로운 column 추가
func = lambda x : x.max() - x.min()

In [18]:
sampleDF["최대-최소"] = sampleDF.apply(func, axis = 1)

In [19]:
sampleDF

,A,B,B,D,최대-최소
2021-03-02,5,1,8,1,7
2021-03-03,5,4,2,8,6
2021-03-04,3,5,0,9,9
2021-03-05,3,6,3,4,3
2021-03-06,7,6,3,9,6
2021-03-07,0,4,4,5,5


### embark_town의 문자열 갯수를 별도의 컬럼인 embark_len 컬럼으로 추가하고 싶다면?


In [26]:
datasets['embark_len'] = datasets['embark_town'].apply(lambda x : len(str(x)))

In [27]:
datasets[['embark_town', 'embark_len']]

,embark_town,embark_len
0,Southampton,11
1,Cherbourg,9
2,Southampton,11
3,Southampton,11
4,Southampton,11
...,...,...
886,Southampton,11
887,Southampton,11
888,Southampton,11
889,Cherbourg,9


- age 피처를 이용하여 나이가 20세 미만이면 child 그렇지 않으면 adult로 구분하여 child_adult 컬럼을 추가한다면?


In [29]:
datasets['child_adult'] = datasets['age'].apply(lambda x : 'child' if x<20 else 'adult')

In [30]:
datasets[['age','child_adult']]

,age,child_adult
0,22.0,adult
1,38.0,adult
2,26.0,adult
3,35.0,adult
4,35.0,adult
...,...,...
886,27.0,adult
887,19.0,child
888,NaN,adult
889,26.0,adult


In [32]:
datasets['child_adult_02'] = datasets.apply(lambda df : 'child' if df.age<20 else 'adult', axis = 1)

In [33]:
datasets[['age','child_adult_02']]

,age,child_adult_02
0,22.0,adult
1,38.0,adult
2,26.0,adult
3,35.0,adult
4,35.0,adult
...,...,...
886,27.0,adult
887,19.0,child
888,NaN,adult
889,26.0,adult


- 15세 이하는 child ,  15~60 까지는 adult , 61 이상은 elderly 로 분류를해서 age_division 컬럼에 저장
- lambda 식으로 apply함수를 이용해서 구현한다면?


In [44]:
datasets['child_adult_detail'] = datasets.apply(lambda df : 'child' if df.age<=15  else ("adult" if df.age<61 else "elderly"),axis=1)

In [46]:
datasets[['age','child_adult_detail']]

,age,child_adult_detail
0,22.0,adult
1,38.0,adult
2,26.0,adult
3,35.0,adult
4,35.0,adult
...,...,...
886,27.0,adult
887,19.0,adult
888,NaN,elderly
889,26.0,adult


In [50]:
def get_age_cate(age) :
    cate =  ""
    if age <=5 :
        cate = "baby"
    elif age <= 12 :
        cate = "child"
    elif age <=24 :
        cate = "student"
    elif age <= 39 :
        cate = "young adult"
    elif age <= 60 :
        cate ="adult"
    else :
        cate ="elderly"
    return cate

In [52]:
datasets['age_cate'] = datasets['age'].apply(lambda age : get_age_cate(age))
datasets['age_cate'].value_counts()

young adult    274
student        208
elderly        199
adult          141
baby            44
child           25
Name: age_cate, dtype: int64

- 승객에 대한 나이(age)와 성별(sex)에 따른 카테고리를 'age/gender' 추가한다
- 조건 1 : 20살이 넘으면 성별을 그대로 사용하고
- 조건 2 : 20살 미만이면 성별에 관계없이 'child' 라고 정의하고 결측값이 있는 경우는 'i don't know'정의한다

In [75]:
condition01 = lambda df : df.sex if df.age >=20 else "child"
condition02 = lambda df : "i don't know" if pd.isnull(df.age) else condition01(df)

In [76]:
datasets['age/gender'] = datasets.apply(condition02 , axis =1 )
datasets['age/gender'].value_counts()
datasets[['age','age/gender']]

,age,age/gender
0,22.0,male
1,38.0,female
2,26.0,female
3,35.0,female
4,35.0,male
...,...,...
886,27.0,male
887,19.0,child
888,NaN,i don't know
889,26.0,male


- fillna : NaN 값을 원하는 값으로 변경

In [87]:
sampleDF.iloc[2,2] = np.nan
sampleDF.iloc[3,0] = np.nan

In [91]:
sampleDF

,A,B,C,D
2021-03-02,7.0,6,6.0,2
2021-03-03,4.0,2,7.0,1
2021-03-04,6.0,6,NaN,7
2021-03-05,NaN,3,5.0,4
2021-03-06,2.0,4,3.0,7
2021-03-07,9.0,0,0.0,5


In [101]:
sampleDF.fillna(0).astype(int)

,A,B,C,D
2021-03-02,7,6,6,2
2021-03-03,4,2,7,1
2021-03-04,6,6,0,7
2021-03-05,0,3,5,4
2021-03-06,2,4,3,7
2021-03-07,9,0,0,5


- 타이타닉 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이값을 대체

In [109]:
datasets['age'] = datasets['age'].fillna(datasets['age'].mean()).astype(int)

In [110]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 19 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            891 non-null int32
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
embark_len     891 non-null int64
child_adult    891 non-null object
age_cate       891 non-null object
age/gender     891 non-null object
dtypes: bool(2), category(2), float64(1), int32(1), int64(5), object(8)
memory usage: 105.0+ KB


- age_gender_category 컬럼 추가하고자 한다
- 조건 1 : 성별(male female)
- 조건 2 : 성별을 나타내는 문자열 뒤에 나이를 추가하고자 한다
- 조건 3 : 예시> male27

In [125]:
datasets['gender+age'] = datasets['sex'] + datasets['age'].astype(str)
datasets.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,embark_len,child_adult,age_cate,age/gender,gender+age
0,0,3,male,22,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,11,adult,student,male,male22
1,1,1,female,38,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,9,adult,young adult,female,female38
2,1,3,female,26,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,11,adult,young adult,female,female26
3,1,1,female,35,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,11,adult,young adult,female,female35
4,0,3,male,35,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,11,adult,young adult,male,male35


In [ ]:
np.random.seed(100)

In [133]:
idxDF = pd.DataFrame(np.vstack([list('ABCDE'), np.round(np.random.rand(3,5),2)]).T,
                    columns = ['col01','col02','col03','col04'])
idxDF

,col01,col02,col03,col04
0,A,0.54,0.66,0.78
1,B,0.3,0.25,0.78
2,C,0.11,0.64,0.61
3,D,0.31,0.2,0.31
4,E,0.46,0.66,0.7


In [135]:
# set_index
idxDF02 = idxDF.set_index('col01')
idxDF02

,col02,col03,col04
col01,,,
A,0.54,0.66,0.78
B,0.3,0.25,0.78
C,0.11,0.64,0.61
D,0.31,0.2,0.31
E,0.46,0.66,0.7


In [137]:
# reset_index
idxDF02.reset_index()

,col01,col02,col03,col04
0,A,0.54,0.66,0.78
1,B,0.3,0.25,0.78
2,C,0.11,0.64,0.61
3,D,0.31,0.2,0.31
4,E,0.46,0.66,0.7


In [138]:
# drop
idxDF02.reset_index(drop=True)

,col02,col03,col04
0,0.54,0.66,0.78
1,0.3,0.25,0.78
2,0.11,0.64,0.61
3,0.31,0.2,0.31
4,0.46,0.66,0.7


### 실습

In [139]:
data = {
    '국어' : [50, 65, 87, 10, 100, 89, 32, 83, 100, 58],
    '영어' : [100, 30, 20, 11, 88, 97, 100, 100, 10, 57],
    '수학' : [100, 59, 88, 29, 99, 29, 22, 99, 29, 50]
}

#### 10명의 학생의 국어, 영어, 수학 점수를 나타내는 데이터프레임을 만든다.
#### 1. 
- 학생 이름을 나타내는 열을 포함시키지 않고 
- 데이터프레임 df_score1 을 생성한 후, 
- df_score1.index 속성에 학생 이름을 나타내는 열을 지정하여 인덱스를 지정한다. 
- reset_index 명령으로 이 인덱스 열을 명령으로 
- 일반 데이터열로 바꾸여 데이터프레임 df_score2을 만든다.

In [141]:
df_score1 = pd.DataFrame(data)
df_score1

,국어,영어,수학
0,50,100,100
1,65,30,59
2,87,20,88
3,10,11,29
4,100,88,99
5,89,97,29
6,32,100,22
7,83,100,99
8,100,10,29
9,58,57,50


In [144]:
df_score1.index=['정호','정섭','보라','수경','지윤','은상','중현','우환','은경','윤수']

In [146]:
df_score1.index.name = "이름"

In [147]:
df_score1

,국어,영어,수학
이름,,,
정호,50,100,100
정섭,65,30,59
보라,87,20,88
수경,10,11,29
지윤,100,88,99
은상,89,97,29
중현,32,100,22
우환,83,100,99
은경,100,10,29


In [149]:
df_score2 = df_score1.reset_index()
df_score2

,이름,국어,영어,수학
0,정호,50,100,100
1,정섭,65,30,59
2,보라,87,20,88
3,수경,10,11,29
4,지윤,100,88,99
5,은상,89,97,29
6,중현,32,100,22
7,우환,83,100,99
8,은경,100,10,29
9,윤수,58,57,50


#### 2.
- 학생 이름을 나타내는 열이 일반 데이터 열을 포함하는 데이터프레임 df_score2에 
- set_index 명령을 적용하여 다시 학생 이름을 나타내는 열을 인덱스로 변경한다.


In [150]:
df_score3 = df_score2.set_index('이름')
df_score3

,국어,영어,수학
이름,,,
정호,50,100,100
정섭,65,30,59
보라,87,20,88
수경,10,11,29
지윤,100,88,99
은상,89,97,29
중현,32,100,22
우환,83,100,99
은경,100,10,29


### Frame Merge
- merge()
- concat()
- join()

In [5]:
df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']},
                    index=[0, 1, 2, 3])
 
df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']},
                    index=[2, 3, 4, 5])

In [6]:
df1

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [7]:
df2

,a,b,c,d
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5


- concat

In [13]:
result1 = pd.concat([df1, df2])
result1

C:\Users\82102\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,a0,b0,c0,NaN
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5


In [10]:
# ignore_index = True
# 인덱스 재조정
result2 = pd.concat([df1,df2], ignore_index=True)
result2

C:\Users\82102\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,a,b,c,d
0,a0,b0,c0,NaN
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
4,a2,b2,c2,d2
5,a3,b3,c3,d3
6,a4,b4,c4,d4
7,a5,b5,c5,d5


In [11]:
result3 = pd.concat([df1,df2],axis=1)
result3

,a,b,c,a,b,c,d
0,a0,b0,c0,NaN,NaN,NaN,NaN
1,a1,b1,c1,NaN,NaN,NaN,NaN
2,a2,b2,c2,a2,b2,c2,d2
3,a3,b3,c3,a3,b3,c3,d3
4,NaN,NaN,NaN,a4,b4,c4,d4
5,NaN,NaN,NaN,a5,b5,c5,d5


- join

In [14]:
# inner join
result4 = pd.concat([df1,df2],axis=1, join="inner")
result4

,a,b,c,a,b,c,d
2,a2,b2,c2,a2,b2,c2,d2
3,a3,b3,c3,a3,b3,c3,d3


- merge

In [15]:
data1 = {
    '학번' : [1,2,3,4],
    '이름' : ['섭섭해' , '김한준' , '김선림' , '최호진'],
    '학년' : [2,4,1,3]
}

data2 = {
    '학번' : [1,2,4,5],
    '학과' : ['CS' , 'MATH' , 'MATH' , 'CS'],
    '학점' : [2.4, 4.5 , 3.4, 4.2]
}

In [16]:
stuDF = pd.DataFrame(data1)
majDF = pd.DataFrame(data2)

In [17]:
stuDF

,학번,이름,학년
0,1,섭섭해,2
1,2,김한준,4
2,3,김선림,1
3,4,최호진,3


In [18]:
majDF

,학번,학과,학점
0,1,CS,2.4
1,2,MATH,4.5
2,4,MATH,3.4
3,5,CS,4.2


In [21]:
# merge -> inner join이 Default값
# on = key
# how = inner | left | right | outer 
pd.merge(stuDF,majDF)

,학번,이름,학년,학과,학점
0,1,섭섭해,2,CS,2.4
1,2,김한준,4,MATH,4.5
2,4,최호진,3,MATH,3.4


In [22]:
pd.merge(stuDF,majDF, on="학번", how="inner")

,학번,이름,학년,학과,학점
0,1,섭섭해,2,CS,2.4
1,2,김한준,4,MATH,4.5
2,4,최호진,3,MATH,3.4


In [23]:
pd.merge(stuDF,majDF, on="학번", how="left")

,학번,이름,학년,학과,학점
0,1,섭섭해,2,CS,2.4
1,2,김한준,4,MATH,4.5
2,3,김선림,1,NaN,NaN
3,4,최호진,3,MATH,3.4


In [24]:
# 공통 인덱스가 없는 경우
data1 = {
    '학번' : [1,2,3,4],
    '이름' : ['섭섭해' , '김한준' , '김선림' , '최호진'],
    '학년' : [2,4,1,3]
}

data2 = {
    '과목코드' : [1,2,4,5],
    '학과' : ['CS' , 'MATH' , 'MATH' , 'CS'],
    '학점' : [2.4, 4.5 , 3.4, 4.2]
}

In [25]:
stuDF = pd.DataFrame(data1)
majDF = pd.DataFrame(data2)

In [28]:
# 학번과 과목코드를 join key 로 설정
pd.merge(stuDF,majDF, left_on="학번", right_on = "과목코드", how="inner")

,학번,이름,학년,과목코드,학과,학점
0,1,섭섭해,2,1,CS,2.4
1,2,김한준,4,2,MATH,4.5
2,4,최호진,3,4,MATH,3.4


### 데이터 불러와서 실습

In [32]:
stock1 = pd.read_excel('./data/stock price.xlsx')
stock1

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000
1,130960,CJ E&M,58540.666667,98900
2,138250,엔에스쇼핑,14558.666667,13200
3,139480,이마트,239230.833333,254500
4,142280,녹십자엠에스,468.833333,10200
5,145990,삼양사,82750.000000,82000
6,185750,종근당,40293.666667,100500
7,192400,쿠쿠홀딩스,179204.666667,177500
8,199800,툴젠,-2514.333333,115400
9,204210,모두투어리츠,3093.333333,3475


In [33]:
stock2 = pd.read_excel('./data/stock valuation.xlsx')
stock2

,id,name,eps,bps,per,pbr
0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,136480,하림,274.166667,3551,11.489362,0.887074
2,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750,종근당,3990.333333,40684,25.185866,2.470259
8,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


- 데이터 프레임 합치기 - 교집합
- 데이터 프레임 합치기 - 합집합
- 데이터 프레임 합치기 - 왼쪽 데이터프레임을 기준, 키 값 분리
- 데이터 프레임 합치기 - 오른쪽 데이터프레임을 기준, 키 값 분리

- 불리언 인덱싱과 결합하여 원하는 데이터 찾아보자

In [34]:
# 교집합
pd.merge(stock1,stock2)

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [35]:
# 합집합
pd.merge(stock1, stock2, how="outer")

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400.0,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359


In [37]:
# 왼쪽 데이터프레임을 기준, 키 값 분리
pd.merge(stock1, stock2,left_on="stock_name",right_on="name", how="left")

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900,130960.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500,139480.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000,145990.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500,185750.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475,204210.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359


In [38]:
# 오른쪽 데이터프레임을 기준, 키 값 분리
pd.merge(stock1, stock2,left_on="stock_name",right_on="name", how="right")

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,130960.0,CJ E&M,58540.666667,98900.0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480.0,이마트,239230.833333,254500.0,139480,이마트,18268.166667,295780,13.931338,0.860437
2,145990.0,삼양사,82750.000000,82000.0,145990,삼양사,5741.000000,108090,14.283226,0.758627
3,185750.0,종근당,40293.666667,100500.0,185750,종근당,3990.333333,40684,25.185866,2.470259
4,204210.0,모두투어리츠,3093.333333,3475.0,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
5,NaN,NaN,NaN,NaN,136480,하림,274.166667,3551,11.489362,0.887074
6,NaN,NaN,NaN,NaN,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
7,NaN,NaN,NaN,NaN,161390,한국타이어,5648.500000,51341,7.453306,0.820007
8,NaN,NaN,NaN,NaN,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
9,NaN,NaN,NaN,NaN,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [40]:
# 불리언 인덱싱과 결합하여 원하는 데이터 찾아보자(price가 50000 미만인 데이터만)
price = stock1[stock1['price'] < 50000]
price

,id,stock_name,value,price
2,138250,엔에스쇼핑,14558.666667,13200
4,142280,녹십자엠에스,468.833333,10200
9,204210,모두투어리츠,3093.333333,3475


- 컬럼 인덱스가 아닌 행 인덱스를 기준으로 병합한다면?
- left_index, right_index

In [49]:
popDF01 = pd.DataFrame({
    'city' : ['seoul','seoul','seoul','busan','busan'],
    'year' : [2010,2005,2021,2018,2015],
    'pop'  : [1234567,2345678,3456789,4567890,5678901]
})
popDF01

,city,year,pop
0,seoul,2010,1234567
1,seoul,2005,2345678
2,seoul,2021,3456789
3,busan,2018,4567890
4,busan,2015,5678901


In [47]:
popDF02 = pd.DataFrame(np.arange(12).reshape((6,2)),
                       index = [['busan','busan','seoul','seoul','seoul','seoul'],
                             [2010,2005,2021,2018,2015,2010]],
                             columns = ['col01','col02'])
                           
popDF02

col01  col02
busan 2010      0      1
      2005      2      3
seoul 2021      4      5
      2018      6      7
      2015      8      9
      2010     10     11

In [50]:
pd.merge(popDF01, popDF02, left_on=['city','year'], right_index=True)

,city,year,pop,col01,col02
0,seoul,2010,1234567,10,11
2,seoul,2021,3456789,4,5


### 그룹분석 및 피봇
- 분할(split)  : 데이터를 특정 조건에 의해 분할
- 적용(apply)  : 데이터를 집계, 변환, 필터링하는데 필요한 함수를 적용
- 결합(combine): 적용 단계의 처리결과를 하나로 결합

In [51]:
df = pd.DataFrame({ "학과" : ["컴퓨터","체육교육과","컴퓨터","체육교육과","컴퓨터"],
                    "학년" : [1, 2, 3, 2, 3],
                    "이름" : ["홍길동","김연아","최길동","아이유","신사임당"],
                    "학점" : [1.5, 4.4, 3.7, 4.5, 3.8]})
df

,학과,학년,이름,학점
0,컴퓨터,1,홍길동,1.5
1,체육교육과,2,김연아,4.4
2,컴퓨터,3,최길동,3.7
3,체육교육과,2,아이유,4.5
4,컴퓨터,3,신사임당,3.8


In [53]:
# grouping -> groupby()
# 학과를 기준으로 grouping

dept_series = df['학과'].groupby(df['학과'])
dept_series

In [58]:
# get_group()
dept_series.get_group('체육교육과')

1    체육교육과
3    체육교육과
Name: 학과, dtype: object

In [61]:
dept_series.value_counts()

# 같은 코드
dept_series.size()

학과
체육교육과    2
컴퓨터      3
Name: 학과, dtype: int64

In [65]:
dept_series.groups

{'체육교육과': Int64Index([1, 3], dtype='int64'),
 '컴퓨터': Int64Index([0, 2, 4], dtype='int64')}

In [69]:
dept = df.groupby(df['학과'])
dept


In [67]:
# dept.mean()
df.groupby(df['학과']).mean()

,학년,학점
학과,,
체육교육과,2.000000,4.45
컴퓨터,2.333333,3.00


In [73]:
dept.agg([np.mean, np.sum])

학년        학점     
           mean sum  mean  sum
학과                            
체육교육과  2.000000   4  4.45  8.9
컴퓨터    2.333333   7  3.00  9.0

In [75]:
# 타이타닉 데이터
row_data = sns.load_dataset('titanic' , engine='python')

In [76]:
row_data_subset_05 = row_data.loc[ :  , ['age' , 'sex' , 'class' , 'fare' , 'survived']] 
row_data_subset_05

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
...,...,...,...,...,...
886,27.0,male,Second,13.0000,0
887,19.0,female,First,30.0000,1
888,NaN,female,Third,23.4500,0
889,26.0,male,First,30.0000,1


In [77]:
print('승객수 : ' , len(row_data_subset_05) )

승객수 :  891


In [78]:
# class 열을 기준으로 분할
class_grp = row_data_subset_05.groupby(['class'])
class_grp 

In [79]:
for key , group in class_grp :
    print('key : ' , key)
    print('number : ' , len(group))
    print(group.head())
    print('\n')

key :  First
number :  216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


key :  Second
number :  184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


key :  Third
number :  491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




In [80]:
class_grp.mean()

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [81]:
class_grp_third = class_grp.get_group('Third')
class_grp_third

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0
...,...,...,...,...,...
882,22.0,female,Third,10.5167,0
884,25.0,male,Third,7.0500,0
885,39.0,female,Third,29.1250,0
888,NaN,female,Third,23.4500,0


In [82]:
# class 열과 sex을 기준으로 분할
class_sex_grp = row_data_subset_05.groupby(['class', 'sex'])
class_sex_grp

In [83]:
for key , group in class_sex_grp :
    print('key : ' , key)
    print('number : ' , len(group))
    print(group.head())
    print('\n')

key :  ('First', 'female')
number :  94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1


key :  ('First', 'male')
number :  122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0


key :  ('Second', 'female')
number :  76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1


key :  ('Second', 'male')
number :  108
     age   sex   class  fare  survived
17   NaN  male 